In [1]:
import utils

	
 !"#%&'()*+,-./0123456789:<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}ĆćčđŠšžНУабвгдезиклмнопрстухчшђјњћ…⁠✋❤﻿🏻🏼🏽👋👌👍👽💪😀😂😃😍😟🙂🙌🙏🤑🤘🤙🤝🤢🤣🤮🥰🥷🥹🧠🫂🫶
159
Step 0: train loss 5.1892, val loss 5.1862
Step 1000: train loss 2.0882, val loss 2.2481
Step 2000: train loss 2.0847, val loss 2.2168
Step 3000: train loss 2.0919, val loss 2.2292
Step 4000: train loss 2.0747, val loss 2.2170
Step 5000: train loss 2.0855, val loss 2.2350
Step 6000: train loss 2.0789, val loss 2.2306
Step 7000: train loss 2.0955, val loss 2.2503
Step 8000: train loss 2.0774, val loss 2.2148
Step 9000: train loss 2.0745, val loss 2.2153


In [2]:
utils.intuition_test('de si sta radis')

de si sta radisa stori p n: 1/24/208/217/4 ta 4 d n:3 ha p un: u
[10202023 dan e ra Fizn:1024110286 b 2023 koo Fiva
